In [221]:
import os
import tqdm
import pickle
import numpy as np
import pandas as pd
import os.path as p
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import AutoTokenizer
import json
from datasets import load_from_disk, Sequence, Value, Features, DatasetDict, Dataset

from fuzzywuzzy import fuzz
from konlpy.tag import Mecab
from pororo import Pororo

In [222]:
mecab = Mecab()

In [237]:
data_path = "/opt/ml/input/checkpoint/ATIREBM25_PREDICT/"
nbest_name = "nbest_predictions_test"
prediction_name = "predictions_test"
save_name = prediction_name + "_revised"

In [238]:
with open(p.join(data_path, nbest_name + ".json"), "r") as f:
    nbest = json.load(f)

In [239]:
q_a_pairs = dict.fromkeys(nbest)

In [240]:
# 만약 모든 후보군이 걸러진다면 top1_answer를 정답에 다시 넣어줌
top1_answer = ''

# 높은 순(nbest 내에 정렬된 순)으로 answer를 검사한 뒤 통과한다면 is_final_answer flog를 True로 바꾸어서 loop 종료
is_final_answer = False

for q_id, qa_candidates in nbest.items():
    is_final_answer = False
    
    for idx, qa_candidate in enumerate(qa_candidates):
        
        if is_final_answer:
            break        
        else:
            answer_candidate = qa_candidate['text'].strip()
            if answer_candidate == 'empty':
                answer_candidate = ''

            if idx == 0:
                top1_answer = answer_candidate
                
            # 문장 중간에 '.'이 들어가는 경우를 피하기 위해, "다."를 기준으로 문장을 나눈 뒤 마지막에 다시 "다."를 추가시켜줌
            sentences = [text.strip().replace("\n","").replace("\\n","") + "다." for text in qa_candidate['context'].split("다.")]
            
            for sentence in sentences:
                if answer_candidate == '':
                    pass
                elif answer_candidate in sentence:
                    #답이 한 단어일 경우
                    if len(answer_candidate.strip().split()) == 1:

                        # mecab으로 pos를 구해서 pos가 명사인지 확인
                        for word, pos in mecab.pos(sentence):
                            # 단어와 형태소가 일치하는 경우
                            if answer_candidate == word:
                                # 명사, 숫자, 외래어 남겨놓기
                                if pos[0] == 'N' or pos[0:2] == 'SN' or pos[0:2] == 'SL':
                                    q_a_pairs[q_id] = answer_candidate
                                    is_final_answer = True
                                    break
                                # 관형사, 부사, 감탄사 등 나머지 제거
                                else:
                                    pass
                            # 단어와 형태소가 일치하지 않는 경우, 첫 번째 형태소만 가지고 판단
                            elif (answer_candidate[0] == word[0]) and (len(word) != len(answer_candidate)):
                                
                                # 만약 답의 첫 번쨰 헝태소가 명사, 숫자, 외래어, 문장부호 등이면 답을 남겨둠
                                if pos[0] == 'N' or pos[0] == 'S' or pos[0:3] == 'XXO' or pos[0:3] == 'XPN':
                                    q_a_pairs[q_id] = answer_candidate
                                    is_final_answer = True
                                    break
                                # 답의 첫 번째 형태소가 조사, 부사, 접미사 등이면 pass
                                elif pos[0] == 'J' or pos[0:2] == 'VC' or pos[0:3] == 'MAG' or pos[0:2] == 'XS' or pos[0] == 'E':
                                    pass
                                else:
                                    q_a_pairs[q_id] = answer_candidate
                                    is_final_answer = True
                                    break 

                        if is_final_answer:
                            break

                    #답이 여러 단어일 경우, 끝이 조사인지 확인하고 조사라면 없애기
                    else:
                        answer_len = len(answer_candidate)
                        for word, pos in mecab.pos(sentence):
                            answer_len -= len(word)
                            if answer_len == 0 and pos[0] == 'J':
                                print(word, pos)
                                q_a_pairs[q_id] = answer_candidate[:len(answer_candidate)-len(word)]
                                is_final_answer = True
                                break
                            else:
                                q_a_pairs[q_id] = answer_candidate
                                is_final_answer = True
                                break
                        if is_final_answer:
                            break
                                
    if not is_final_answer:
        q_a_pairs[q_id] = top1_answer

In [241]:
q_a_pairs

{'mrc-1-000653': '필수',
 'mrc-1-001113': '여전히',
 'mrc-0-002191': '대통령인 빌헬름 미클라스',
 'mrc-0-003951': '뉴질랜드',
 'mrc-1-001272': '프랑스군',
 'mrc-1-000993': '체내 기관',
 'mrc-0-005021': '구체제',
 'mrc-1-000163': '최소화하여 수성에 접근하기 위해서는 필수',
 'mrc-0-001283': '순조 11년(1811)',
 'mrc-0-004543': '고전도성 철',
 'mrc-0-000439': '이미',
 'mrc-0-002895': '칼라치 전방 약250km 지점',
 'mrc-0-000535': '롭 포드',
 'mrc-1-001724': '노스햄프턴 교회',
 'mrc-0-000901': '언니',
 'mrc-0-001606': '바실리카식',
 'mrc-0-000266': '최소한',
 'mrc-0-001326': '‘자유’',
 'mrc-0-000032': '국방부 보고서',
 'mrc-0-005215': '입석대(立石臺)',
 'mrc-0-005407': '숙의 정씨',
 'mrc-0-003683': '담임교사',
 'mrc-0-003644': '이탈리아',
 'mrc-0-002835': '망자',
 'mrc-0-000049': '이미',
 'mrc-1-001829': '최소한',
 'mrc-1-001662': '시라카와 성 공방전',
 'mrc-0-001206': '2003년',
 'mrc-0-004007': '신시내티대학교',
 'mrc-1-000418': '유달산 꽃축제(구. 유달산 개나리축제)',
 'mrc-0-003133': '생물학자',
 'mrc-0-004646': '건늠선',
 'mrc-0-001058': '집에서 집 없는 곳',
 'mrc-0-002361': '영토',
 'mrc-0-004830': '13개',
 'mrc-0-002762': '경복사(景福寺)',
 'mrc-0-000395': '

In [242]:
with open(p.join(data_path, prediction_name + ".json"), "r") as f:
    predictions = json.load(f)

In [243]:
difference = []
for question in predictions:
    if predictions[question] != q_a_pairs[question]:
        difference.append((question, predictions[question], [q_a_pairs[question]]))
        print('question: ', question, '\t', 'previous answer: ', predictions[question], '\t', 'after post-processing: ', q_a_pairs[question])

print(len(difference))

question:  mrc-0-000439 	 previous answer:  꼭 	 after post-processing:  이미
question:  mrc-0-000901 	 previous answer:  무조건 	 after post-processing:  언니
question:  mrc-0-000266 	 previous answer:  쉽사리 	 after post-processing:  최소한
question:  mrc-0-000049 	 previous answer:  왜냐하면 	 after post-processing:  이미
question:  mrc-1-001829 	 previous answer:  쉽사리 	 after post-processing:  최소한
question:  mrc-0-001206 	 previous answer:  간혹 	 after post-processing:  2003년
question:  mrc-0-001058 	 previous answer:  이미 	 after post-processing:  집에서 집 없는 곳
question:  mrc-0-004413 	 previous answer:  가끔 	 after post-processing:  필수
question:  mrc-0-000624 	 previous answer:  이미 	 after post-processing:  노르웨이
question:  mrc-0-001725 	 previous answer:  이따금 	 after post-processing:  '호주인 조니'
question:  mrc-0-000854 	 previous answer:  반드시 	 after post-processing:  1976년
question:  mrc-1-001523 	 previous answer:  다소 	 after post-processing:  니 기븐
question:  mrc-0-003896 	 previous answer:  왜냐하면 	 after

In [244]:
# JSON 파일로 저장
prediction_file = os.path.join(
            data_path, save_name + ".json"
        )

with open(prediction_file, "w") as writer:
    writer.write(json.dumps(q_a_pairs, indent=4, ensure_ascii=False) + "\n")